In [2]:
from inside_ns import enter_ns
from orbit3d import get_3dtrajectory
import numpy as np
import mpmath as mp
from matplotlib import pyplot as plt
from orbit2d import get_2dtrajectory
from scipy.optimize import root_scalar

In [4]:
def get_2dtraj_in_ns(rin: mp.matrix, vin: mp.matrix, tin: float, k: float, radius: float) -> mp.matrix:
    omega = mp.sqrt(k / radius**3)
    rns = lambda t: mp.matrix([rin[0]*mp.cos(omega*(t-tin)) + vin[0]/omega*mp.sin(omega*(t-tin)),
                            rin[1]*mp.cos(omega*(t-tin)) + vin[1]/omega*mp.sin(omega*(t-tin))])

    vns = lambda t: mp.matrix([vin[0]*mp.cos(omega*(t-tin)) - rin[0]*omega*mp.sin(omega*(t-tin)),
                            vin[1]*mp.cos(omega*(t-tin)) - rin[1]*omega*mp.sin(omega*(t-tin))])

    return rns, vns

In [6]:
def plot_2dtraj_in_ns(rin: mp.matrix, vin: mp.matrix, tin: float, k: float, radius: float, n_points: int = 10_000) -> mp.matrix:
    omega = mp.sqrt(k / radius**3)
    rns, _ = get_2dtraj_in_ns(rin, vin, tin, k, R)

    tsns = mp.linspace(tin, tin + 2*mp.pi/omega, n_points)
    rsns = np.array([rns(tns) for tns in tsns])

    plt.plot(rsns.T[0], rsns.T[1])

In [7]:
def find_ns_exit(rin: mp.matrix, vin: mp.matrix, tin: float, k: float, radius: float, n_points: int = 1_000) -> tuple:
    omega = mp.sqrt(k / radius**3)
    r2d, v2d = get_2dtraj_in_ns(rin, vin, tin, k, radius)
    to_root = lambda angle: mp.norm(r2d(angle/omega+tin)) - radius

    angles = mp.linspace(0, 2*mp.pi, n_points, endpoint=False)
    to_root_vals = np.array([to_root(angle) for angle in angles])
    index = np.where(to_root_vals[:-1]*to_root_vals[1:] < 0)[0][0]

    angle_exit = root_scalar(to_root, bracket=(0, angles[index+1])).root

    return r2d(angle_exit/omega+tin), v2d(angle_exit/omega+tin), angle_exit/omega+tin

In [ ]:
def get_2dtraj_through_ns(d0: float, b: float, v0: float, k: float, radius: float) -> tuple:
    infall = get_2dtrajectory(d0, b, v0, k)
    thetain, r2din, rdotin, thetadotin, tin = enter_ns(d0, b, v0, k)

    rin = mp.matrix([r2din*mp.cos(thetain), r2din*mp.sin(thetain)])
    vin = mp.matrix([rdotin*mp.cos(thetain) - r2din*thetadotin*mp.sin(thetain),
                    rdotin*mp.sin(thetain) + r2din*thetadotin*mp.cos(thetain)])

    rout, vout, tout = find_ns_exit(rin, vin, tin, k, radius)

    v0out = mp.norm(vout)
    bout = np.cross(rout, vout) / v0out
    d0out = mp.sqrt(mp.norm(rout)**2 - bout**2)

    outfall = get_2dtrajectory(d0out, bout, v0out, k, t0=tout)

    return (thetain, tout), infall, outfall

    